In [33]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from spot_loader import load_spot
from hfc_loader import load_hfc
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [34]:
path_benedrien = r'M:\PRICING\Pricer - Agrégation\benedrien\Dev Python\Benedri2 v3.11 python-actioned.xlsm'
path_parameters = r'M:\PRICING\Pricer - Agrégation\benedrien\Dev Python\Parameters.xlsx'

Get input

In [35]:

sheet_input = 'INPUT'
df_raw = pd.read_excel(path_benedrien, sheet_name=sheet_input, header=None)

rows = {
    6: 'Power',
    7: 'Région',
    9: 'Nom client',
    10: 'Nom site',
    13: 'Path',
    14: 'CARD',
    15: 'Date début de relève',
    16: 'Date fin de relève',
    19: 'Date début de livraison',
    20: 'Date fin de livraison +1j',
    21: 'Marge',
    22: 'Premium supplémentaire',
    23: 'Délai de paiement (jours)',
}

row_sites = df_raw.iloc[11, 2:]
last_col = row_sites.last_valid_index()
end_col = last_col if pd.notna(last_col) else df_raw.shape[1] - 1

subset = df_raw.loc[list(rows.keys()), 2:end_col]

subset.index = [rows[i] for i in subset.index]

df_input = subset.T.reset_index(drop=True)



In [36]:
start_date_histo = df_input["Date début de relève"].min()
end_date_histo = df_input["Date fin de relève"].max()

spot_histo = load_spot(start_date_histo, end_date_histo)

hfc = load_hfc(datetime(2025,10,28))

Getting sites' production and make hourly calculations

In [37]:
site_dfs_hourly = {}

for idx, site in df_input.iterrows():
    site_name = site['Nom site']
    site_path = site['Path']


    df_site = pd.read_excel(site_path, header=None, usecols=[0, 1, 8])
    df_site.columns = ['Date', 'Hour', 'Prod (kWh)']




    df_site['Datetime'] = pd.to_datetime(
    df_site['Date'].astype(str).str.strip() + ' ' +
    df_site['Hour'].astype(str).str.replace('h', ':').str.zfill(5),
    format='%Y-%m-%d %H:%M',
    errors='coerce'
)


    df_site = df_site[['Datetime', 'Prod (kWh)']]

    df_site = pd.merge(df_site, spot_histo[['Datetime', 'Spot']], on='Datetime', how='left')

    df_site['Prod sans HPSN (kWh)'] = df_site.apply(
        lambda x: x['Prod (kWh)'] if pd.notna(x['Spot']) and x['Spot'] >= 0 else 0,
        axis=1
    )

    df_site['Valo réelle (€)'] = df_site['Prod (kWh)'] * df_site['Spot']/1000
    df_site['Valo hors HPSN (€)'] = df_site['Prod sans HPSN (kWh)'] * df_site['Spot']/1000

    site_dfs_hourly[site_name] = df_site




Monthly

In [38]:
site_dfs_monthly_past = {}

for site_name, df_hourly in site_dfs_hourly.items():
    df_hourly['Datetime'] = pd.to_datetime(df_hourly['Datetime'])
    df_hourly['Année'] = df_hourly['Datetime'].dt.year
    df_hourly['Mois'] = df_hourly['Datetime'].dt.month


    monthly_groups = df_hourly.groupby(['Année', 'Mois'])

    monthly_data = []

    for (year, month), group in monthly_groups:
        sum_prod = group['Prod (kWh)'].sum()
        sum_prod_hpsn = group['Prod sans HPSN (kWh)'].sum()
        avg_spot = group['Spot'].mean()
        sum_valo_reel = (group['Prod (kWh)'] * group['Spot']).sum()
        sum_valo_hpsn = (group['Prod sans HPSN (kWh)'] * group['Spot']).sum()

        cr_reel = sum_valo_reel / (sum_prod * avg_spot) if sum_prod and avg_spot else None
        cr_hpsn = sum_valo_hpsn / (sum_prod_hpsn * avg_spot) if sum_prod_hpsn and avg_spot else None

        monthly_data.append({
            'Année': year,
            'Mois': month,
            'Prod (kWh)': sum_prod,
            'Prod sans HPSN (kWh)': sum_prod_hpsn,
            'Valo réelle (€)': sum_valo_reel / 1000,        
            'Valo hors HPSN (€)': sum_valo_hpsn / 1000,
            'Spot': avg_spot,
            'CR réel': cr_reel,
            'CR hors HPSN': cr_hpsn
        })


    df_monthly = pd.DataFrame(monthly_data)
    site_dfs_monthly_past[site_name] = df_monthly


CR région et HPSN

In [39]:
xls = pd.ExcelFile(path_parameters)
all_sheets = xls.sheet_names

region_CR_dfs = {}

month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12,
    'Janv': 1, 'Févr': 2, 'Mars': 3, 'Avr': 4, 'Mai': 5, 'Juin': 6,
    'Juil': 7, 'Août': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Déc': 12
}

for idx, site in df_input.iterrows():
    site_name = site['Nom site']
    region_prefix = str(site['Région'])[:6]  

    matching_sheets = [s for s in all_sheets if s.startswith("Eolien_") and region_prefix in s]
    sheet_name = matching_sheets[0]  

    df_CR_region = pd.read_excel(path_parameters, sheet_name=sheet_name)

    df_CR_region = df_CR_region.melt(id_vars=['Mois'], var_name='Année', value_name='CR région')
    df_CR_region['Mois'] = df_CR_region['Mois'].map(month_map).astype(int)
    df_CR_region.sort_values(['Année', 'Mois'], inplace=True)
    df_CR_region.reset_index(drop=True, inplace=True)
    df_CR_region['CR région'] = df_CR_region['CR région']/100

    region_CR_dfs[sheet_name] = df_CR_region




    df_nb_hpsn = pd.read_excel(path_parameters, sheet_name="Nb HPSN")

    df_nb_hpsn = df_nb_hpsn.melt(id_vars=['Mois'], var_name='Année', value_name='Nb HPSN')
    df_nb_hpsn['Mois'] = df_nb_hpsn['Mois'].map(month_map).astype(int)
    df_nb_hpsn.sort_values(['Année', 'Mois'], inplace=True)
    df_nb_hpsn.reset_index(drop=True, inplace=True)

    df_monthly = site_dfs_monthly_past[site_name]
    df_monthly = pd.merge(df_monthly, df_CR_region, on=['Année', 'Mois'], how='left')
    df_monthly = pd.merge(df_monthly, df_nb_hpsn, on=['Année', 'Mois'], how='left')

    df_monthly = df_monthly[df_monthly['CR région'].notna()].copy()

    df_monthly['Delta CR (Parc - région)'] = df_monthly['CR hors HPSN'] - df_monthly['CR région']

    df_monthly['Prod corrigée (kWh)'] = df_monthly['Prod sans HPSN (kWh)'] + 0.25*df_monthly['Nb HPSN']*site['Power']*1000

    site_dfs_monthly_past[site_name] = df_monthly


Monthly averages

In [40]:
site_dfs_averages = {}

for site_name, df_monthly in site_dfs_monthly_past.items():


    df_avg = (
        df_monthly
        .groupby('Mois', as_index=False)
        .agg({
            'Prod corrigée (kWh)': 'mean',
            'Delta CR (Parc - région)': 'mean'
        })
    )


    df_avg.rename(columns={
        'Prod corrigée (kWh)': 'Prod moyenne corrigée (kWh)',
        'Delta CR (Parc - région)': 'Delta CR moyen (Parc - région)'
    }, inplace=True)


    df_avg.sort_values('Mois', inplace=True)


    site_dfs_averages[site_name] = df_avg

In [41]:
site_dfs_monthly_proj = {}

for idx, site in df_input.iterrows():
    site_name = site['Nom site']
    region_prefix = str(site['Région'])[:6]
    matching_sheets = [s for s in region_CR_dfs.keys() if s.startswith("Eolien_") and region_prefix in s]
    sheet_name = matching_sheets[0]
    df_CR_region = region_CR_dfs[sheet_name]

    df_avg = site_dfs_averages[site_name]

    start_date = pd.to_datetime(site['Date début de livraison'])
    end_date = pd.to_datetime(site['Date fin de livraison +1j'])
    months = pd.date_range(start=start_date, end=end_date - pd.Timedelta(days=1), freq='MS')

    df_proj = pd.DataFrame({
        'Année': [m.year for m in months],
        'Mois': [m.month for m in months]
    })

    df_proj = pd.merge(df_proj, hfc, on=['Année', 'Mois'], how='left')
    df_proj = pd.merge(df_proj, df_CR_region[['Année', 'Mois', 'CR région']], on=['Année', 'Mois'], how='left')
    df_proj = pd.merge(df_proj, df_avg[['Mois', 'Prod moyenne corrigée (kWh)', 'Delta CR moyen (Parc - région)']], on='Mois', how='left')
    df_proj = pd.merge(df_proj, df_nb_hpsn[['Année', 'Mois', 'Nb HPSN']], on=['Année', 'Mois'], how='left')

    df_proj['CR région projeté'] = df_proj['CR région']
    df_proj['CR parc projeté'] = df_proj['CR région projeté'] + df_proj['Delta CR moyen (Parc - région)']
    df_proj.rename(columns={'Prod moyenne corrigée (kWh)': 'Production projetée (kWh)'}, inplace=True)

    df_proj['Perte de prod (HPSN) (kWh)'] = -0.25 * site['Power'] * 1000 * df_proj['Nb HPSN']

    df_proj['Production projetée finale (kWh)'] = df_proj['Production projetée (kWh)'] + df_proj['Perte de prod (HPSN) (kWh)']


    df_proj.drop(['Delta CR moyen (Parc - région)', 'Nb HPSN'], axis=1, inplace=True)

    df_proj.sort_values(['Année', 'Mois'], inplace=True)
    df_proj.reset_index(drop=True, inplace=True)

    site_dfs_monthly_proj[site_name] = df_proj



Calculate Gain shape

In [42]:
start_date_proj = df_input["Date début de livraison"].min()
end_date_proj = df_input["Date fin de livraison +1j"].max() - timedelta(days=1)


all_years = range(start_date_proj.year, end_date_proj.year + 1)

gain_shape = pd.DataFrame(
    index=df_input['Nom site'].unique(),
    columns=all_years
)

for site_name in df_input['Nom site'].unique():
    df_proj = site_dfs_monthly_proj[site_name]

    for year in all_years:
        df_y = df_proj[df_proj['Année'] == year]

        if len(df_y) == 0:
            gain_shape.loc[site_name, year] = None
            continue

        num = (df_y['Hfc'] * df_y['Production projetée finale (kWh)']).sum()
        den = df_y['Production projetée finale (kWh)'].sum()
        avg_hfc_year = df_y['Hfc'].mean()

        gain_shape.loc[site_name, year] = (num / den) / avg_hfc_year if den != 0 and avg_hfc_year != 0 else None


In [43]:
CR_quarterly_parcs_dfs = {}

for site_name, df_proj in site_dfs_monthly_proj.items():
    df_proj['Trimestre'] = ((df_proj['Mois'] - 1) // 3 + 1)

    quarterly_data = []
    grouped = df_proj.groupby(['Année', 'Trimestre'])
    for (year, quarter), group in grouped:
        sum_prod = group['Production projetée finale (kWh)'].sum()
        avg_hfc = group['Hfc'].mean()
        weighted_sum = (group['Hfc'] * group['CR parc projeté'] * group['Production projetée finale (kWh)']).sum()
        CR_quarter = weighted_sum / (avg_hfc * sum_prod) if sum_prod != 0 and avg_hfc != 0 else None
        quarterly_data.append({
            'Année': year,
            'Trimestre': quarter,
            'Production finale (kWh)': sum_prod,
            'CR Quarter': CR_quarter
        })

    df_quarterly = pd.DataFrame(quarterly_data)

    annual_data = []
    grouped_year = df_proj.groupby('Année')
    for year, group in grouped_year:
        sum_prod = group['Production projetée finale (kWh)'].sum()
        avg_hfc = group['Hfc'].mean()
        weighted_sum = (group['Hfc'] * group['CR parc projeté'] * group['Production projetée finale (kWh)']).sum()
        CR_year = weighted_sum / (avg_hfc * sum_prod) if sum_prod != 0 and avg_hfc != 0 else None
        annual_data.append({'Année': year, 'CR annuel': CR_year})

    df_annual = pd.DataFrame(annual_data)
    df_quarterly = df_quarterly.merge(df_annual, on='Année', how='left')

    CR_quarterly_parcs_dfs[site_name] = df_quarterly
    


Insert in the excel

Calculation of premiums

In [44]:

df_par_raw = pd.read_excel(path_parameters, sheet_name='Parameters', header=None)

cell_map = {
    'flex offre profil + spot (protection 70% volume)': 'C3',
    'flex offre indexé+ spot pondéré (protection 80% volume)': 'C4',
    'flex indexe': 'C5',
    'écarts variable': 'C8',
    'écarts fixe': 'C9',
    'wacc': 'C12',
    'pourcentage flex fixe': 'C16',
    'taxe organique':'C19',
    'premium supp': 'C22',
    'frais opérationnels (par an par site)': 'C33',
    'garantie maison mère': 'C35',
    'spread bid-ask': 'C37'
}

def excel_cell_to_iloc(cell):
    """Convert Excel cell reference (e.g., C3) into iloc row/column indices."""
    col = ord(cell[0].upper()) - ord('A')
    row = int(cell[1:]) - 1
    return row, col


params = {}
for label, cell in cell_map.items():
    r, c = excel_cell_to_iloc(cell)
    params[label] = df_par_raw.iat[r, c]

df_parameters = pd.DataFrame([params])



In [45]:

premium_columns = [
    'Total premium + marge (A soustraire au coeff a)',
    'Premium délai de paiement',
    'Premium taxe organique',
    'Premium spread bid/Ask',
    'Premium écarts (fixe)',
    'Frais opérationnels',
    'Frais garantie maison mère',
    'Premium flex fixe',
    'Premium canibalisation (sécurité)',
    'Premium flex (variable)',
    'Premium écart (variable)',
]

hfc_fwd = (df_proj['Hfc']).mean()

P50_dict = {}

for site_name, df_avg in site_dfs_averages.items():
    P50_value = df_avg['Prod moyenne corrigée (kWh)'].mean()
    P50_dict[site_name] = P50_value




df_premiums = pd.DataFrame({
    'Nom site': df_input['Nom site'],
    'Marge' : df_input['Marge']

})

P50_series = df_premiums['Nom site'].map(P50_dict)


for col in premium_columns:
    df_premiums[col] = None


df_premiums['Premium délai de paiement'] = -(
    df_input['Délai de paiement (jours)'] - 20
) * (
    df_parameters.loc[0, 'wacc'] / 365
) * hfc_fwd


df_premiums['Premium taxe organique'] = (
    df_parameters.loc[0, 'taxe organique'] * hfc_fwd
)


df_premiums['Premium spread bid/Ask'] = (
    df_parameters.loc[0, 'spread bid-ask']
)


df_premiums['Premium écarts (fixe)'] = (
    df_parameters.loc[0, 'écarts fixe']
)


df_premiums['Frais opérationnels'] = (np.where(
    P50_series > 1000000,
    df_parameters.loc[0, 'frais opérationnels (par an par site)'] * 1000 / P50_series,
    df_parameters.loc[0, 'frais opérationnels (par an par site)'] / P50_series
))/10


df_premiums['Frais garantie maison mère'] = (
    df_parameters.loc[0, 'garantie maison mère']
)


df_premiums['Premium flex fixe'] = (
    df_parameters.loc[0, 'flex indexe'] * (1 - df_parameters.loc[0, 'pourcentage flex fixe'])
)


df_premiums['Premium canibalisation (sécurité)'] = 0.01


df_premiums['Premium flex (variable)'] = (
    df_parameters.loc[0, 'flex indexe'] / hfc_fwd * df_parameters.loc[0, 'pourcentage flex fixe']
)


df_premiums['Premium écart (variable)'] = (
    df_parameters.loc[0, 'écarts variable']
)

df_premiums['Total premium (%)'] = (
    df_premiums['Premium canibalisation (sécurité)'] +
    df_premiums['Premium flex (variable)'] +
    df_premiums['Premium écart (variable)']
)

other_prems = [
    'Premium délai de paiement',
    'Premium taxe organique',
    'Premium spread bid/Ask',
    'Premium écarts (fixe)',
    'Frais opérationnels',
    'Frais garantie maison mère',
    'Premium flex fixe',
    'Total premium (%)'
]

df_premiums['Total premium + marge (A soustraire au coeff a)'] = (
    df_premiums[other_prems].sum(axis=1) + df_premiums['Marge']
)




In [46]:

quarter_dates = pd.date_range(start=start_date_proj, end=end_date_proj, freq='QS')

df_a_b = pd.MultiIndex.from_product(
    [df_input['Nom site'].unique(), quarter_dates],
    names=['Nom site', 'Quarter Start']
).to_frame(index=False)

df_a_b['Year'] = df_a_b['Quarter Start'].dt.year
df_a_b['Quarter'] = df_a_b['Quarter Start'].dt.quarter
df_a_b.drop(columns='Quarter Start', inplace=True)


whole_year_rows = pd.MultiIndex.from_product(
    [df_input['Nom site'].unique(), all_years],
    names=['Nom site', 'Year']
).to_frame(index=False)
whole_year_rows['Quarter'] = 'Whole year'

df_a_b = pd.concat([df_a_b, whole_year_rows], ignore_index=True)
df_a_b.sort_values(by=['Nom site', 'Year', 'Quarter'], inplace=True)
df_a_b.reset_index(drop=True, inplace=True)

extra_columns = [
    'Volume (MWh)',
    'Nombre d\'heures',
    'valeur profil annuel (% du CAL BL)',
    'Prix éléctron période sur HfC',
    'Capture Rate  Bleq valeur (MW)',
    'a',
    'b'
]

df_a_b['Volume (MWh)'] = None
df_a_b['Nombre d\'heures'] = None
df_a_b['Capture Rate'] = None
df_a_b['Bleq valeur (MW)'] = None
df_a_b['valeur profil annuel (% du CAL BL)'] = None
df_a_b['Prix éléctron période sur HfC'] = None
df_a_b['a'] = None
df_a_b['Prix éléctron période sur HfC'] = None

for idx, row in df_a_b.iterrows():
    site_name = row['Nom site']
    year = row['Year']
    quarter = row['Quarter']

    df_proj = site_dfs_monthly_proj[site_name]
    df_cr = CR_quarterly_parcs_dfs[site_name]


    A_premium = df_premiums.loc[df_premiums['Nom site'] == site_name,
                                'Total premium + marge (A soustraire au coeff a)'].iloc[0]

    total_premium_pct = df_premiums.loc[df_premiums['Nom site'] == site_name,
                                        'Total premium (%)'].iloc[0]

    if quarter == 'Whole year':
        mask = df_proj['Année'] == year
        volume_mwh = df_proj[mask]['Production projetée finale (kWh)'].sum() / 1000
        nb_heures = df_proj[mask]['nb_heures'].sum()

        cr_row = df_cr[df_cr['Année'] == year]
        CR_val = cr_row['CR annuel'].iloc[0] if len(cr_row) else None

        prix_hfc = df_proj[mask]['Hfc'].mean()
        profil_annuel = gain_shape.loc[site_name, year] if year in gain_shape.columns else None

    else:
        mask = (df_proj['Année'] == year) & (df_proj['Trimestre'] == quarter)
        volume_mwh = df_proj[mask]['Production projetée finale (kWh)'].sum() / 1000
        nb_heures = df_proj[mask]['nb_heures'].sum()

        cr_row = df_cr[(df_cr['Année'] == year) & (df_cr['Trimestre'] == quarter)]
        CR_val = cr_row['CR Quarter'].iloc[0] if len(cr_row) else None

        prix_hfc = df_proj[mask]['Hfc'].mean()
        profil_annuel = None

    if nb_heures and CR_val is not None and nb_heures != 0:
        bleq = volume_mwh * CR_val / nb_heures
    else:
        bleq = None

 
    a_val = A_premium
    b_val = CR_val - total_premium_pct if CR_val is not None else None

    df_a_b.at[idx, 'Volume (MWh)'] = volume_mwh
    df_a_b.at[idx, 'Nombre d\'heures'] = nb_heures
    df_a_b.at[idx, 'Capture Rate'] = CR_val
    df_a_b.at[idx, 'Prix éléctron période sur HfC'] = prix_hfc
    df_a_b.at[idx, 'valeur profil annuel (% du CAL BL)'] = profil_annuel
    df_a_b.at[idx, 'Bleq valeur (MW)'] = bleq
    df_a_b.at[idx, 'a'] = -a_val
    df_a_b.at[idx, 'b'] = b_val




Insert all in excel

In [47]:
wb = openpyxl.load_workbook(path_benedrien, keep_vba=True)

created_sheets = []

for site_name in site_dfs_monthly_past.keys():
    df_monthly = site_dfs_monthly_past[site_name]
    df_avg = site_dfs_averages[site_name]
    df_proj = site_dfs_monthly_proj[site_name]
    df_quarterly = CR_quarterly_parcs_dfs[site_name]

    sheet_name = f"site_{site_name}"[:31]

    if sheet_name in wb.sheetnames:
        ws_old = wb[sheet_name]
        wb.remove(ws_old)

    ws = wb.create_sheet(sheet_name)
    created_sheets.append(sheet_name)

    for r_idx, row in enumerate(dataframe_to_rows(df_monthly, index=False, header=True), start=1):
        for c_idx, value in enumerate(row, start=1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    start_col = df_monthly.shape[1] + 3

    for r_idx, row in enumerate(dataframe_to_rows(df_avg, index=False, header=True), start=1):
        for c_idx, value in enumerate(row, start=start_col):
            ws.cell(row=r_idx, column=c_idx, value=value)
    
    start_col += df_avg.shape[1] + 3

    for r_idx, row in enumerate(dataframe_to_rows(df_proj, index=False, header=True), start=1):
        for c_idx, value in enumerate(row, start=start_col):
            ws.cell(row=r_idx, column=c_idx, value=value)

    start_col += df_proj.shape[1] + 3

    for r_idx, row in enumerate(dataframe_to_rows(df_quarterly, index=False, header=True), start=1):
        for c_idx, value in enumerate(row, start=start_col):
            ws.cell(row=r_idx, column=c_idx, value=value)

for sheet_name in reversed(created_sheets):
    for sheet_name in reversed(created_sheets):
        wb.move_sheet(wb[sheet_name], offset=-wb.index(wb[sheet_name]))




ws_out = wb['OUTPUT PYTHON']
start_col = 4
site_names = df_input['Nom site'].tolist()

for idx_site, site_name in enumerate(site_names):
    col = start_col + idx_site
    idx_input = df_input.index[df_input['Nom site'] == site_name][0]

    ws_out.cell(row=10, column=col, value=df_input.at[idx_input, 'Nom client'])
    ws_out.cell(row=11, column=col, value=df_input.at[idx_input, 'Nom site'])
    ws_out.cell(row=14, column=col, value=df_input.at[idx_input, 'CARD'])
    ws_out.cell(row=15, column=col, value=df_input.at[idx_input, 'Date début de relève'])
    ws_out.cell(row=16, column=col, value=df_input.at[idx_input, 'Date fin de relève'])
    ws_out.cell(row=26, column=col, value=df_input.at[idx_input, 'Date début de livraison'])
    ws_out.cell(row=27, column=col, value=df_input.at[idx_input, 'Date fin de livraison +1j'])
    ws_out.cell(row=28, column=col, value=df_input.at[idx_input, 'Marge'])

    fixed_rows = {
        32: 'Total premium + marge (A soustraire au coeff a)',
        33: 'Premium délai de paiement',
        34: 'Premium taxe organique',
        35: 'Premium spread bid/Ask',
        36: 'Premium écarts (fixe)',
        37: 'Frais opérationnels',
        38: 'Frais garantie maison mère',
        39: 'Premium flex fixe',
        40: 'Marge'
    }
    for r, col_name in fixed_rows.items():
        ws_out.cell(row=r, column=col, value=df_premiums.at[idx_input, col_name])

    variable_rows = {
        43: 'Total premium (%)',
        44: 'Premium canibalisation (sécurité)',
        45: 'Premium flex (variable)',
        46: 'Premium écart (variable)'
    }
    for r, col_name in variable_rows.items():
        ws_out.cell(row=r, column=col, value=df_premiums.at[idx_input, col_name])

    for idx_year, year in enumerate(all_years):
        row_base = 50 + 58 * idx_year
        ws_out.cell(row=row_base, column=2, value=year)

        df_wy = df_a_b[(df_a_b['Nom site'] == site_name) &
                       (df_a_b['Year'] == year) &
                       (df_a_b['Quarter'] == 'Whole year')]
        if not df_wy.empty:
            ws_out.cell(row=row_base+1, column=col, value=df_wy['Volume (MWh)'].values[0])
            ws_out.cell(row=row_base+2, column=col, value=df_wy['Nombre d\'heures'].values[0])
            ws_out.cell(row=row_base+3, column=col, value=df_wy['valeur profil annuel (% du CAL BL)'].values[0])
            ws_out.cell(row=row_base+4, column=col, value=df_wy['Prix éléctron période sur HfC'].values[0])
            ws_out.cell(row=row_base+5, column=col, value=df_wy['Capture Rate'].values[0])
            ws_out.cell(row=row_base+7, column=col, value=df_wy['Bleq valeur (MW)'].values[0])
            ws_out.cell(row=row_base+8, column=col, value=df_wy['a'].values[0])
            ws_out.cell(row=row_base+9, column=col, value=df_wy['b'].values[0])

        quarter_offsets = [12, 23, 34, 45]
        for q_idx, q_offset in enumerate(quarter_offsets, start=1):
            df_q = df_a_b[(df_a_b['Nom site'] == site_name) &
                          (df_a_b['Year'] == year) &
                          (df_a_b['Quarter'] == q_idx)]
            if not df_q.empty:
                ws_out.cell(row=row_base + q_offset, column=col, value=df_q['Volume (MWh)'].values[0])
                ws_out.cell(row=row_base + q_offset + 2, column=col, value=df_q['Nombre d\'heures'].values[0])


                avg_hfc = df_q['Prix éléctron période sur HfC'].values[0]
                a_val = df_q['a'].values[0]
                b_val = df_q['b'].values[0]

                avg_price_formula = avg_hfc * b_val + a_val


                ws_out.cell(row=row_base + q_offset + 3, column=col, value=avg_price_formula)
                ws_out.cell(row=row_base + q_offset + 4, column=col, value=df_wy['Prix éléctron période sur HfC'].values[0])
                ws_out.cell(row=row_base + q_offset + 5, column=col, value=df_q['Capture Rate'].values[0])
                ws_out.cell(row=row_base + q_offset + 7, column=col, value=df_q['Bleq valeur (MW)'].values[0])
                ws_out.cell(row=row_base + q_offset + 8, column=col, value=df_q['a'].values[0])
                ws_out.cell(row=row_base + q_offset + 9, column=col, value=df_q['b'].values[0])

wb.save(path_benedrien)



